In [11]:
import requests

# Найти спутник с максимальным количеством передатчиков

In [12]:
sat_transmitters = dict()
page = 1
transmitters = requests.get(f'https://sonik.space/api/transmitters/?page={page}').json()
while isinstance(transmitters, list):
    for item in transmitters:
        key = f"{item['satellite_name']}/{item['satellite_norad']}"
        if key not in sat_transmitters.keys():
            sat_transmitters[key] = 1
        else:
            sat_transmitters[key] += 1
    page += 1
    transmitters = requests.get(f'https://sonik.space/api/transmitters/?page={page}').json()

In [13]:
satellite, max_transmitters = max(sat_transmitters.items(), key=lambda x: x[1])
sat_name, sat_norad = satellite.rsplit('/', 1)
print(f'Спутник с максимальным количеством передатчиков: {sat_name}; NORAD ID {sat_norad}')
print(f'Количество передатчиков: {max_transmitters}')

Спутник с максимальным количеством передатчиков: ISS; NORAD ID 25544
Количество передатчиков: 40


# У найденного спутника найти передатчик, у которого больше всего (в процентном соотношении) успешных наблюдений.

In [14]:
sat_transmitters = dict()
page = 1
transmitters = requests.get(f'https://sonik.space/api/transmitters/?page={page}&satellite__norad_cat_id={sat_norad}').json()
while isinstance(transmitters, list):
    for item in transmitters:
        key = item['description']
        if 'stat' in item.keys():
            success_count = item['stat']['good_count']
            total_count = item['stat']['total_count']
        else:
            success_count, total_count = 0, 0
        if total_count == 0:
            percentage = 0
        else:
            percentage = round(success_count / total_count, 4)*100

        sat_transmitters[key] = percentage

    page += 1
    transmitters = requests.get(f'https://sonik.space/api/transmitters/?page={page}&satellite__norad_cat_id={sat_norad}').json()

In [15]:
transmitter, percentage = max(sat_transmitters.items(), key=lambda x: x[1])
print(f'Передатчик с максимальным процентом успешных наблюдений: {transmitter}. Процент успешных наблюдений: {percentage}')

Передатчик с максимальным процентом успешных наблюдений: Mode V/U FM - Voice Repeater. Процент успешных наблюдений: 75.76


# Выведем информацию о найденном спутнике и найденном передатчике в стандартный поток ввода/вывода.

In [18]:
def print_satellite_data(data):
    print(f"Satellite ID: {data.get('sat_id', 'N/A')}")
    print(f"NORAD: {data.get('norad', 'N/A')}")
    print(f"Name: {data.get('name', 'N/A')}")
    print(f"Alternate Names: {data.get('names', 'N/A')}")
    print(f"Call sign: {data.get('names', 'N/A')}")
    print(f"Status: {data.get('status', 'N/A')}")
    print(f"Launch Date: {data.get('launched', 'N/A')}")
    print(f"Decayed: {data.get('decayed', 'N/A')}")
    print(f"Operator: {data.get('operator', 'N/A')}")
    print(f"Network: {data.get('network', 'N/A')}")
    print(f"Countries: {data.get('countries', 'N/A')}")
    print(f"Website: {data.get('website', 'N/A')}")
    print(f"Frequency Violator: {'Yes' if data.get('is_frequency_violator', False) else 'No'}")
    print(f"Image: {data.get('image', 'N/A')}")
    print(f"Unknown: {'Yes' if data.get('unknown', False) else 'No'}")
    print(f"Citation: {data.get('citation', 'N/A')}")
    print(f"Created On: {data.get('created', 'N/A')}")
    print(f"Created By: {data.get('created_by', 'N/A')}")
    print("-" * 40)

def print_transmitter_data(data):
    print(f"Transmitter UUID: {data.get('uuid', 'N/A')}")
    print(f"Description: {data.get('description', 'N/A')}")
    print(f"Satellite Name: {data.get('satellite_name', 'N/A')}")
    print(f"Satellite NORAD: {data.get('satellite_norad', 'N/A')}")
    print(f"Satellite Launch Date: {data.get('satellite_launch', 'N/A')}")
    print(f"Type: {data.get('type', 'N/A')}")
    print(f"Status: {data.get('status', 'N/A')}")
    print(f"Uplink Frequency: {data.get('uplink_low', 'N/A')} Hz")
    print(f"Downlink Frequency: {data.get('downlink_low', 'N/A')} Hz")
    print(f"Uplink Mode: {data.get('uplink_mode', 'N/A')}")
    print(f"Downlink Mode: {data.get('downlink_mode', 'N/A')}")
    print(f"Frequency Inversion: {'Yes' if data.get('invert', False) else 'No'}")
    print(f"Baud Rate: {data.get('baud', 'N/A')}")
    print(f"Citation: {data.get('citation', 'N/A')}")
    print(f"Approved: {'Yes' if data.get('approved', False) else 'No'}")
    print(f"Unconfirmed: {'Yes' if data.get('unconfirmed', False) else 'No'}")
    print(f"Unknown: {'Yes' if data.get('unknown', False) else 'No'}")
    print("Statistics:")
    stats = data.get('stat', {})
    print(f"  Future Count: {stats.get('future_count', 'N/A')}")
    print(f"  Total Count: {stats.get('total_count', 'N/A')}")
    print(f"  Unknown Count: {stats.get('unknown_count', 'N/A')}")
    print(f"  Good Count: {stats.get('good_count', 'N/A')}")
    print(f"  Bad Count: {stats.get('bad_count', 'N/A')}")
    print(f"  Failed Count: {stats.get('failed_count', 'N/A')}")
    print(f"  Transmitter UUID: {stats.get('transmitter_uuid', 'N/A')}")
    print("-" * 40)

sat_data = requests.get(f'https://sonik.space/api/satellites?norad_cat_id={sat_norad}').json()[0]
print_satellite_data(sat_data)

transmitter_data = requests.get(f'https://sonik.space/api/transmitters?description={transmitter}').json()[0]
print_transmitter_data(transmitter_data)


Satellite ID: XSKZ-5603-1870-9019-3066
NORAD: 25544
Name: ISS
Alternate Names: ZARYA
Call sign: ZARYA
Status: alive
Launch Date: 1998-11-20T00:00:00Z
Decayed: None
Operator: None
Network: None
Countries: RU,US
Website: https://www.nasa.gov/mission_pages/station/main/index.html
Frequency Violator: No
Image: http://sonik.space/media/satellites/ISS_CA2pT3O.jpeg
Unknown: No
Citation: 
Created On: 2023-11-22T14:01:41.430814Z
Created By: None
----------------------------------------
Transmitter UUID: eozSf5mKyzNxoascs8V4bV
Description: Mode V/U FM - Voice Repeater
Satellite Name: ISS
Satellite NORAD: 25544
Satellite Launch Date: None
Type: Передатчик
Status: active
Uplink Frequency: 145990000 Hz
Downlink Frequency: 437800000 Hz
Uplink Mode: FM
Downlink Mode: FM
Frequency Inversion: No
Baud Rate: None
Citation: https://www.ariss.org/press-releases/september-2-2020
Approved: Yes
Unconfirmed: No
Unknown: No
Statistics:
  Future Count: 35
  Total Count: 957
  Unknown Count: 34
  Good Count: 725
